In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
import json
import requests
import time
from datetime import datetime


In [2]:
#Random Latitudes
lats = np.random.uniform(-90, 90, 1500)
#Random Longitudes
lngs = np.random.uniform(-180, 180, 1500)

#combined
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [25]:
# # Use the tuple() function to display the latitude and longitude combinations.
# for coordinate in coordinates:
#     print(coordinate[0], coordinate[1])

In [26]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

616

In [ ]:
print(cities)

In [ ]:
#Create an empty list to hold the weather data
city_data = []
#Print the beginning of the logging
print("Beginnging Data Retrieval      ")
print("-------------------------------")

#Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
    
    #Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >=50):
        set_count += 1
        record_count = 1
        time.sleep(60)
        
    # Create endpoint URL for each city.
    city_url = url + "&q=" + city.replace(" ", "+")
        
    # Log the URL,record, and set number and the city.
    print(f"Processing {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                            "Lat": city_lat,
                            "Lng": city_lng,
                            "Max Temp": city_max_temp,
                            "Humidity": city_humidity,
                            "Cloudiness": city_clouds,
                            "Wind Speed": city_wind,
                            "Country": city_country,
                            "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [46]:
# Convert the array of dictionaries to Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,New Norfolk,AU,2021-11-24 18:44:28,-42.7826,147.0587,55.92,63,9,3.00
1,Kavaratti,IN,2021-11-24 18:44:28,10.5669,72.6420,80.92,81,100,5.70
2,Sobolevo,RU,2021-11-24 18:44:29,54.4299,31.8996,33.51,91,95,13.78
3,Ushuaia,AR,2021-11-24 18:42:47,-54.8000,-68.3000,49.66,53,40,21.85
4,Yellowknife,CA,2021-11-24 18:44:29,62.4560,-114.3525,-5.71,70,75,9.22
5,Jamestown,US,2021-11-24 18:42:24,42.0970,-79.2353,44.71,33,100,4.00
6,East London,ZA,2021-11-24 18:44:30,-33.0153,27.9116,61.74,72,20,5.75
7,Oranjestad,AW,2021-11-24 18:41:00,12.5240,-70.0270,84.15,70,40,14.97
8,Barrow,US,2021-11-24 18:44:30,71.2906,-156.7887,-2.18,71,75,11.50
9,Victoria,HK,2021-11-24 18:39:38,22.2855,114.1577,67.24,76,0,4.00


In [50]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")